# REMOVE DUPLICATE

In [25]:
"""make merged tsv"""
import glob
import pandas as pd
import re

if False:

    pfixes = ["data/MMS/org/tid-10142-actives", "data/MMS/org/tid-137-actives", "data/MMS/org/tid-194-actives", "data/MMS/org/tid-8-actives"]

    for pfix in pfixes:
        files = glob.glob(pfix+"*.tsv")
        dfs = []
        out_dfs_ng = dict()
        out_dfs_ok = dict()
        for file in files:
            df = pd.read_csv(file, delimiter="\t", index_col=0)
            df["file"] = file
            dfs.append(df)
            print(df.shape, file)

        res = None
        for df in dfs:
            if res is None:
                res = df
            else:
                res = pd.concat([res, df], axis=0)

        # print(list(res.duplicated(subset="aromatic_smiles").values))

        for _idx, _val in enumerate(list(res.duplicated(subset="aromatic_smiles").values)):
            _entry = res.iloc[_idx,:]
            _file = res.iloc[_idx,:].loc['file']
            if _val:
                # print(f"{list(res.index)[_idx]}: {res.iloc[_idx,:].loc[['chembl-id', 'file']]}, {res.iloc[_idx,:].loc['chembl-id']}")
                if _file not in out_dfs_ng.keys():
                    out_dfs_ng[_file] = pd.DataFrame(columns=list(res.columns)[:-1])
                    out_dfs_ng[_file].loc[_entry.name] = _entry.iloc[:-1]
                else:
                    out_dfs_ng[_file].loc[_entry.name] = _entry.iloc[:-1]
            else:
                if _file not in out_dfs_ok.keys():
                    out_dfs_ok[_file] = pd.DataFrame(columns=list(res.columns)[:-1])
                    out_dfs_ok[_file].loc[_entry.name] = _entry.iloc[:-1]
                else:
                    out_dfs_ok[_file].loc[_entry.name] = _entry.iloc[:-1]
            
                

        print("---- NG ----")
        for _k, _v in out_dfs_ng.items():
            _file = _k.replace("/org/", "/ng/")
            print(_file)
            _v.to_csv(_file, sep="\t", index=True)
            # display(_v)
        
        print("---- OK ----")
        for _k, _v in out_dfs_ok.items():
            _file = _k.replace("/org/", "/")
            print(_file)
            _v.to_csv(_file, sep="\t", index=True)

        # display(_v)
        # res = res.drop_duplicates(subset="chembl-id")
        # display(res)
        # res.to_csv(f"{pfix.replace('-actives', '')}_merged.tsv", sep="\t", index=True)
        # print(list(res.duplicated(subset="aromatic_smiles").values))


# CORRELATION, KMEANS

In [29]:
import glob
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

files = glob.glob("data/MMS/uniq/*.tsv")
dfs = []
out_dfs_ng = dict()
out_dfs_ok = dict()
for file in files:
    df = pd.read_csv(file, delimiter="\t", index_col=0)
    df["file"] = file
    dfs.append(df)
    print(df.shape, file)

res = None
for df in dfs:
    if res is None:
        res = df
    else:
        res = pd.concat([res, df], axis=0)

FEATURES = ["arorings", "a_acc", "a_don", "a_heavy", "logP(o/w)", "RBC", "rings", "TPSA", "vdw_vol", "Weight"]

"""CORRELATION"""
corr = res.loc[:, FEATURES].corr()
display(corr.style.background_gradient())

"""PCA/KMeans"""
res_feat = res.loc[:, FEATURES]
X = res_feat.values
_scaler = StandardScaler()
_scaler.fit(X)
X_std = _scaler.transform(X)
df_std = pd.DataFrame(X_std, index=res_feat.index, columns=res_feat.columns)

pca = PCA(n_components=10)
X_std_T = df_std.T.values
pca.fit(X_std_T)
Xd = pca.transform(X_std_T)

_kmeans = KMeans(n_clusters=4).fit_predict(X_std_T)
for _gp in range(4):
    print(f"{_gp}: {[list(res_feat.columns)[_idx] for _idx, _g in enumerate(_kmeans) if _g == _gp]}")

# print([(list(res_feat.columns)[_idx],_kmeans[_idx]) for _idx in range(len(_kmeans))])

# """CORRELATION"""
# corr = df_std.corr()
# display(corr.style.background_gradient())

# display(df_std.describe())

(56, 24) data/MMS/uniq/tid-10627-actives_106mms_sub-moedescriptors.tsv
(42, 24) data/MMS/uniq/tid-280-actives_417mms_sub-moedescriptors.tsv
(61, 24) data/MMS/uniq/tid-194-actives_232mms_sub-moedescriptors.tsv
(83, 24) data/MMS/uniq/tid-137-actives_49mms_sub-moedescriptors.tsv
(53, 24) data/MMS/uniq/tid-10280-actives_90mms_sub-moedescriptors.tsv
(42, 24) data/MMS/uniq/tid-278-actives_20mms_sub-moedescriptors.tsv
(50, 24) data/MMS/uniq/tid-10142-actives_100mms_sub-moedescriptors.tsv
(42, 24) data/MMS/uniq/tid-129-actives_462mms_sub-moedescriptors.tsv
(51, 24) data/MMS/uniq/tid-20174-actives_26mms_sub-moedescriptors.tsv
(61, 24) data/MMS/uniq/tid-10142-actives_154mms_sub-moedescriptors.tsv
(40, 24) data/MMS/uniq/tid-8-actives_49mms_sub-moedescriptors.tsv
(75, 24) data/MMS/uniq/tid-8-actives_44mms_sub-moedescriptors.tsv


,arorings,a_acc,a_don,a_heavy,logP(o/w),RBC,rings,TPSA,vdw_vol,Weight
arorings,1.000000,-0.271718,-0.139058,0.535633,0.563429,-0.173849,0.660110,-0.203202,0.555511,0.466248
a_acc,-0.271718,1.000000,0.378852,0.150469,-0.594401,0.506803,-0.149662,0.680599,0.073820,0.137700
a_don,-0.139058,0.378852,1.000000,0.015640,-0.354281,0.264626,-0.048907,0.450079,-0.032064,-0.028771
a_heavy,0.535633,0.150469,0.015640,1.000000,0.494230,0.400559,0.644860,0.096036,0.973469,0.932106
logP(o/w),0.563429,-0.594401,-0.354281,0.494230,1.000000,-0.042915,0.483158,-0.630486,0.585848,0.499039
RBC,-0.173849,0.506803,0.264626,0.400559,-0.042915,1.000000,-0.160225,0.256905,0.410981,0.379267
rings,0.660110,-0.149662,-0.048907,0.644860,0.483158,-0.160225,1.000000,-0.192200,0.651502,0.547622
TPSA,-0.203202,0.680599,0.450079,0.096036,-0.630486,0.256905,-0.192200,1.000000,-0.041392,0.092122
vdw_vol,0.555511,0.073820,-0.032064,0.973469,0.585848,0.410981,0.651502,-0.041392,1.000000,0.920076
Weight,0.466248,0.137700,-0.028771,0.932106,0.499039,0.379267,0.547622,0.092122,0.920076,1.000000


0: ['arorings', 'logP(o/w)', 'rings']
1: ['RBC']
2: ['a_heavy', 'vdw_vol', 'Weight']
3: ['a_acc', 'a_don', 'TPSA']


/home/raku/.pyenv/versions/anaconda3-2021.05/envs/figp_env2/lib/python3.9/site-packages/scikit_learn-1.2.0-py3.9-linux-x86_64.egg/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
